In [53]:
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import rich.table
import planetary_computer
from pystac_client import Client
from shapely.geometry import Polygon
import rioxarray as rio
import pystac

from IPython.display import Image
from datacube.utils.cog import write_cog

In [4]:
#we'll use this function to get bounding box coordinates from a list of points 
def points2coords(pt_ls): #should be [xmin, ymin, xmax, ymax]
    
    coords_ls = [(pt_ls[0], pt_ls[1]), (pt_ls[0], pt_ls[3]),
                 (pt_ls[2], pt_ls[3]), (pt_ls[2], pt_ls[1]),
                 (pt_ls[0], pt_ls[1])]
    return coords_ls

In [17]:
time_range = "2024-07-31/2024-08-31"
poly_path = '/media/laserglaciers/upernavik/ziyu_sentinel_1/bounds.gpkg'
df = gpd.read_file(poly_path)
bbox = list(df.bounds.iloc[0])
bbox_coords = points2coords(bbox)
bbox_coords

[(-38.26232395985209, 65.55253151074457),
 (-38.26232395985209, 66.54871027518462),
 (-37.1990624503319, 66.54871027518462),
 (-37.1990624503319, 65.55253151074457),
 (-38.26232395985209, 65.55253151074457)]

In [22]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)
items = search.get_all_items()
print(f"Found: {len(items):d} datasets")

Found: 14 datasets


In [23]:
list(df.bounds.iloc[0])

[-38.26232395985209, 65.55253151074457, -37.1990624503319, 66.54871027518462]

In [24]:
print(type(catalog))
print(type(search))
print(type(items))

<class 'pystac_client.client.Client'>
<class 'pystac_client.item_search.ItemSearch'>
<class 'pystac.item_collection.ItemCollection'>


In [55]:
items[1].id

'S1A_IW_GRDH_1SDH_20240828T090355_20240828T090420_055411_06C222_rtc'

In [26]:
df = gpd.GeoDataFrame.from_features(items.to_dict(), crs='epsg:4326')
df.head(5)

,geometry,datetime,platform,s1:shape,proj:bbox,proj:epsg,proj:shape,end_datetime,constellation,s1:resolution,...,sar:center_frequency,sar:resolution_range,s1:product_timeliness,sar:resolution_azimuth,sar:pixel_spacing_range,sar:observation_direction,sar:pixel_spacing_azimuth,sar:looks_equivalent_number,s1:instrument_configuration_ID,sat:platform_international_designator
0,"POLYGON ((-36.37863 64.35281, -35.65734 65.586...",2024-08-28T09:04:33.388393Z,SENTINEL-1A,"[28561, 21544]","[371570.0, 7111430.0, 657180.0, 7326870.0]",32624,"[21544, 28561]",2024-08-28 09:04:45.887416+00:00,Sentinel-1,high,...,5.405,20,NRT-3h,22,10,right,10,4.4,7,2014-016A
1,"POLYGON ((-35.52465 65.80058, -34.78374 67.063...",2024-08-28T09:04:08.388846Z,SENTINEL-1A,"[28269, 21466]","[404120.0, 7277340.0, 686810.0, 7492000.0]",32624,"[21466, 28269]",2024-08-28 09:04:20.887869+00:00,Sentinel-1,high,...,5.405,20,NRT-3h,22,10,right,10,4.4,7,2014-016A
2,"POLYGON ((-34.16018 64.64852, -33.3608 65.9852...",2024-08-23T08:56:12.711763Z,SENTINEL-1A,"[27977, 20768]","[478650.0, 7162320.0, 758420.0, 7370000.0]",32624,"[20768, 27977]",2024-08-23 08:56:25.211475+00:00,Sentinel-1,high,...,5.405,20,NRT-3h,22,10,right,10,4.4,7,2014-016A
3,"POLYGON ((-33.1158 66.36832, -32.39387 67.4536...",2024-08-23T08:55:47.711588Z,SENTINEL-1A,"[29126, 22765]","[237550.0, 7317640.0, 528810.0, 7545290.0]",32625,"[22765, 29126]",2024-08-23 08:56:00.210549+00:00,Sentinel-1,high,...,5.405,20,NRT-3h,22,10,right,10,4.4,7,2014-016A
4,"POLYGON ((-38.34734 64.52698, -37.70512 65.635...",2024-08-21T09:12:46.122087Z,SENTINEL-1A,"[29554, 22387]","[270360.0, 7113670.0, 565900.0, 7337540.0]",32624,"[22387, 29554]",2024-08-21 09:12:58.620974+00:00,Sentinel-1,high,...,5.405,20,NRT-3h,22,10,right,10,4.4,7,2014-016A


In [27]:
df.columns

Index(['geometry', 'datetime', 'platform', 's1:shape', 'proj:bbox',
       'proj:epsg', 'proj:shape', 'end_datetime', 'constellation',
       's1:resolution', 'proj:transform', 's1:datatake_id', 'start_datetime',
       's1:orbit_source', 's1:slice_number', 's1:total_slices',
       'sar:looks_range', 'sat:orbit_state', 'sar:product_type',
       'sar:looks_azimuth', 'sar:polarizations', 'sar:frequency_band',
       'sat:absolute_orbit', 'sat:relative_orbit', 's1:processing_level',
       'sar:instrument_mode', 'sar:center_frequency', 'sar:resolution_range',
       's1:product_timeliness', 'sar:resolution_azimuth',
       'sar:pixel_spacing_range', 'sar:observation_direction',
       'sar:pixel_spacing_azimuth', 'sar:looks_equivalent_number',
       's1:instrument_configuration_ID',
       'sat:platform_international_designator'],
      dtype='object')

In [28]:
Image(url=items[0].assets["rendered_preview"].href)

In [29]:
table = rich.table.Table('key','value')
for k,v in sorted(items[0].properties.items()):
    table.add_row(k, str(v))
table

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ key                                   ┃ value                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ constellation                         │ Sentinel-1                                                  │
│ datetime                              │ 2024-08-28T09:04:33.388393Z                                 │
│ end_datetime                          │ 2024-08-28 09:04:45.887416+00:00                            │
│ platform                              │ SENTINEL-1A                                                 │
│ proj:bbox                             │ [371570.0, 7111430.0, 657180.0, 7326870.0]                  │
│ proj:epsg                             │ 32624                                                       │
│ proj:shape                            │ [21544, 28561]                                              │
│ proj:transform                        │ [10.0, 0.0, 371570.0, 0.0, -10.0, 7326870.0, 0.0, 0.0, 1.0] │
│ s1:datatake_id                        │ 442914                                                      │
│ s1:instrument_configuration_ID        │ 7                                                           │
│ s1:orbit_source                       │ PREORB                                                      │
│ s1:processing_level                   │ 1                                                           │
│ s1:product_timeliness                 │ NRT-3h                                                      │
│ s1:resolution                         │ high                                                        │
│ s1:shape                              │ [28561, 21544]                                              │
│ s1:slice_number                       │ 3                                                           │
│ s1:total_slices                       │ 10                                                          │
│ sar:center_frequency                  │ 5.405                                                       │
│ sar:frequency_band                    │ C                                                           │
│ sar:instrument_mode                   │ IW                                                          │
│ sar:looks_azimuth                     │ 1                                                           │
│ sar:looks_equivalent_number           │ 4.4                                                         │
│ sar:looks_range                       │ 5                                                           │
│ sar:observation_direction             │ right                                                       │
│ sar:pixel_spacing_azimuth             │ 10                                                          │
│ sar:pixel_spacing_range               │ 10                                                          │
│ sar:polarizations                     │ ['HH', 'HV']                                                │
│ sar:product_type                      │ GRD                                                         │
│ sar:resolution_azimuth                │ 22                                                          │
│ sar:resolution_range                  │ 20                                                          │
│ sat:absolute_orbit                    │ 55411                                                       │
│ sat:orbit_state                       │ descending                                                  │
│ sat:platform_international_designator │ 2014-016A                                                   │
│ sat:relative_orbit                    │ 39                                                          │
│ start_datetime                        │ 2024-08-28 09:04:20.889370+00:00                            │
└───────────────────────────────────────┴───────

In [30]:
items[0].properties

{'datetime': '2024-08-28T09:04:33.388393Z',
 'platform': 'SENTINEL-1A',
 's1:shape': [28561, 21544],
 'proj:bbox': [371570.0, 7111430.0, 657180.0, 7326870.0],
 'proj:epsg': 32624,
 'proj:shape': [21544, 28561],
 'end_datetime': '2024-08-28 09:04:45.887416+00:00',
 'constellation': 'Sentinel-1',
 's1:resolution': 'high',
 'proj:transform': [10.0, 0.0, 371570.0, 0.0, -10.0, 7326870.0, 0.0, 0.0, 1.0],
 's1:datatake_id': '442914',
 'start_datetime': '2024-08-28 09:04:20.889370+00:00',
 's1:orbit_source': 'PREORB',
 's1:slice_number': '3',
 's1:total_slices': '10',
 'sar:looks_range': 5,
 'sat:orbit_state': 'descending',
 'sar:product_type': 'GRD',
 'sar:looks_azimuth': 1,
 'sar:polarizations': ['HH', 'HV'],
 'sar:frequency_band': 'C',
 'sat:absolute_orbit': 55411,
 'sat:relative_orbit': 39,
 's1:processing_level': '1',
 'sar:instrument_mode': 'IW',
 'sar:center_frequency': 5.405,
 'sar:resolution_range': 20,
 's1:product_timeliness': 'NRT-3h',
 'sar:resolution_azimuth': 22,
 'sar:pixel_spa

In [31]:
from dask.distributed import Client

client = Client(processes=False)
print(client.dashboard_link)

http://10.98.4.175:41377/status


/home/laserglaciers/miniconda3/envs/py310v2/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41377 instead
  warnings.warn(


In [32]:
type(items)

pystac.item_collection.ItemCollection

In [35]:
import stackstac
import os

da = stackstac.stack(
    planetary_computer.sign(items), bounds_latlon=bbox, epsg=32624
)

In [36]:
da

<xarray.DataArray 'stackstac-e62be534f02ce365d5bfd72fc7ef6339' (time: 14,
                                                                band: 2,
                                                                y: 11200,
                                                                x: 5041)> Size: 13GB
dask.array<fetch_raster_window, shape=(14, 2, 11200, 5041), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/39)
  * time                                   (time) datetime64[ns] 112B 2024-08...
    id                                     (time) <U66 4kB 'S1A_IW_GRDH_1SDH_...
  * band                                   (band) <U2 16B 'hh' 'hv'
  * x                                      (x) float64 40kB 5.328e+05 ... 5.8...
  * y                                      (y) float64 90kB 7.382e+06 ... 7.2...
    s1:orbit_source                        <U6 24B 'PREORB'
    ...                                     ...
    sar:resolution_azimuth                 int64 8B 22
    sar:looks_range                        int64 8B 5
    description                            (band) <U175 1kB 'Terrain-correcte...
    title                                  (band) <U43 344B 'HH: horizontal t...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32624
Attributes:
    spec:        RasterSpec(epsg=32624, bounds=(532750.0, 7270230.0, 583160.0...
    crs:         epsg:32624
    transform:   | 10.00, 0.00, 532750.00|\n| 0.00,-10.00, 7382230.00|\n| 0.0...
    resolution:  10.0

In [56]:
stac_item = pystac.read_file(f'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/{items[1].id}')

In [57]:
stac_item

<Item id=S1A_IW_GRDH_1SDH_20240828T090355_20240828T090420_055411_06C222_rtc>

In [58]:
single_id = stackstac.stack(
    planetary_computer.sign(stac_item), bounds_latlon=bbox, epsg=32624
)

In [59]:
single_id.sel(band='hh').rio.to_raster('test2.tif')

In [60]:
# da.to_netcdf('test_all.nc',engine='rasterio')

In [75]:
da

<xarray.DataArray 'stackstac-e62be534f02ce365d5bfd72fc7ef6339' (time: 14,
                                                                band: 2,
                                                                y: 11200,
                                                                x: 5041)> Size: 13GB
dask.array<fetch_raster_window, shape=(14, 2, 11200, 5041), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/39)
  * time                                   (time) datetime64[ns] 112B 2024-08...
    id                                     (time) <U66 4kB 'S1A_IW_GRDH_1SDH_...
  * band                                   (band) <U2 16B 'hh' 'hv'
  * x                                      (x) float64 40kB 5.328e+05 ... 5.8...
  * y                                      (y) float64 90kB 7.382e+06 ... 7.2...
    s1:orbit_source                        <U6 24B 'PREORB'
    ...                                     ...
    sar:resolution_azimuth                 int64 8B 22
    sar:looks_range                        int64 8B 5
    description                            (band) <U175 1kB 'Terrain-correcte...
    title                                  (band) <U43 344B 'HH: horizontal t...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32624
Attributes:
    spec:        RasterSpec(epsg=32624, bounds=(532750.0, 7270230.0, 583160.0...
    crs:         epsg:32624
    transform:   | 10.00, 0.00, 532750.00|\n| 0.00,-10.00, 7382230.00|\n| 0.0...
    resolution:  10.0

In [68]:
odc.geo.xr.write_cog(single_id.isel(time=0),fname=f'{items[1].id}.tif',
                       overwrite=True)

PosixPath('S1A_IW_GRDH_1SDH_20240828T090355_20240828T090420_055411_06C222_rtc.tif')

In [67]:
single_id.isel(time=0)

<xarray.DataArray 'stackstac-c3327f9cf15717fa95d53ca3bdc3fccf' (band: 2,
                                                                y: 11200,
                                                                x: 5041)> Size: 903MB
dask.array<getitem, shape=(2, 11200, 5041), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/45)
    time                                   datetime64[ns] 8B 2024-08-28T09:04...
    id                                     <U66 264B 'S1A_IW_GRDH_1SDH_202408...
  * band                                   (band) <U2 16B 'hh' 'hv'
  * x                                      (x) float64 40kB 5.328e+05 ... 5.8...
  * y                                      (y) float64 90kB 7.382e+06 ... 7.2...
    s1:orbit_source                        <U6 24B 'PREORB'
    ...                                     ...
    title                                  (band) <U43 344B 'HH: horizontal t...
    file:size                              (band) int64 16B 1843552679 182538...
    description                            (band) <U175 1kB 'Terrain-correcte...
    file:checksum                          (band) <U32 256B 'b463f6b20b95af1f...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32624
Attributes:
    spec:        RasterSpec(epsg=32624, bounds=(532750.0, 7270230.0, 583160.0...
    crs:         epsg:32624
    transform:   | 10.00, 0.00, 532750.00|\n| 0.00,-10.00, 7382230.00|\n| 0.0...
    resolution:  10.0

In [74]:
single_id.isel(band=0)

<xarray.DataArray 'stackstac-c3327f9cf15717fa95d53ca3bdc3fccf' (time: 1,
                                                                y: 11200,
                                                                x: 5041)> Size: 452MB
dask.array<getitem, shape=(1, 11200, 5041), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/45)
  * time                                   (time) datetime64[ns] 8B 2024-08-2...
    id                                     (time) <U66 264B 'S1A_IW_GRDH_1SDH...
    band                                   <U2 8B 'hh'
  * x                                      (x) float64 40kB 5.328e+05 ... 5.8...
  * y                                      (y) float64 90kB 7.382e+06 ... 7.2...
    s1:orbit_source                        <U6 24B 'PREORB'
    ...                                     ...
    title                                  <U43 172B 'HH: horizontal transmit...
    file:size                              int64 8B 1843552679
    description                            <U175 700B 'Terrain-corrected gamm...
    file:checksum                          <U32 128B 'b463f6b20b95af1f16094e6...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32624
Attributes:
    spec:        RasterSpec(epsg=32624, bounds=(532750.0, 7270230.0, 583160.0...
    crs:         epsg:32624
    transform:   | 10.00, 0.00, 532750.00|\n| 0.00,-10.00, 7382230.00|\n| 0.0...
    resolution:  10.0

In [78]:
for item in items:
    print(item.id)
    stac_item = pystac.read_file(f'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/{item.id}')
    single_id = stackstac.stack(planetary_computer.sign(stac_item), bounds_latlon=bbox, epsg=32624)
    odc.geo.xr.write_cog(single_id.isel(time=0),fname=f'{item.id}.tif',
                       overwrite=True)

S1A_IW_GRDH_1SDH_20240828T090420_20240828T090445_055411_06C222_rtc
S1A_IW_GRDH_1SDH_20240828T090355_20240828T090420_055411_06C222_rtc
S1A_IW_GRDH_1SDH_20240823T085600_20240823T085625_055338_06BF7D_rtc
S1A_IW_GRDH_1SDH_20240823T085535_20240823T085600_055338_06BF7D_rtc
S1A_IW_GRDH_1SDH_20240821T091233_20240821T091258_055309_06BE5F_rtc
S1A_IW_GRDH_1SDH_20240821T091208_20240821T091233_055309_06BE5F_rtc
S1A_IW_GRDH_1SDH_20240816T090420_20240816T090445_055236_06BBAE_rtc
S1A_IW_GRDH_1SDH_20240816T090355_20240816T090420_055236_06BBAE_rtc
S1A_IW_GRDH_1SDH_20240811T085600_20240811T085625_055163_06B90A_rtc
S1A_IW_GRDH_1SDH_20240811T085535_20240811T085600_055163_06B90A_rtc
S1A_IW_GRDH_1SDH_20240809T091233_20240809T091258_055134_06B802_rtc
S1A_IW_GRDH_1SDH_20240809T091208_20240809T091233_055134_06B802_rtc
S1A_IW_GRDH_1SDH_20240804T090420_20240804T090445_055061_06B54A_rtc
S1A_IW_GRDH_1SDH_20240804T090355_20240804T090420_055061_06B54A_rtc


In [80]:
single_id.band

<xarray.DataArray 'band' (band: 2)> Size: 16B
array(['hh', 'hv'], dtype='<U2')
Coordinates: (12/41)
  * band                                   (band) <U2 16B 'hh' 'hv'
    s1:orbit_source                        <U6 24B 'PREORB'
    s1:product_timeliness                  <U6 24B 'NRT-3h'
    s1:shape                               object 8B {21466, 28262}
    proj:transform                         object 8B {0.0, 1.0, 403840.0, 749...
    platform                               <U11 44B 'SENTINEL-1A'
    ...                                     ...
    title                                  (band) <U43 344B 'HH: horizontal t...
    file:size                              (band) int64 16B 1840840923 182755...
    description                            (band) <U175 1kB 'Terrain-correcte...
    file:checksum                          (band) <U32 256B '6503a0ed04489c6f...
    raster:bands                           object 8B {'nodata': -32768, 'data...
    epsg                                   int64 8B 32624